## 1. Preparación de datos

#### Se utiliza como conjunto de datos CIFAR-10, un conjunto de datos comúnmente utilizado para la clasificación de imágenes.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transformaciones de los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Descargar y cargar los datos de entrenamiento y prueba
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


## 2. Definición de modelo, optimizador y criterio de pérdida

In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Definición del modelo CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capa convolucional 1: 32 filtros, kernel 3x3, activación ReLU
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Capa de max pooling 1: tamaño del pool 2x2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 2: 64 filtros, kernel 3x3, activación ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 2: tamaño del pool 2x2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 3: 64 filtros, kernel 3x3, activación ReLU
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 3: tamaño del pool 2x2
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa completamente conectada: 64 unidades, activación ReLU
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # Capa de salida: 10 unidades (para clasificación en 10 clases), activación softmax
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Aplicar la primera capa convolucional seguida de ReLU y pooling
        x = self.pool1(F.relu(self.conv1(x)))
        # Aplicar la segunda capa convolucional seguida de ReLU y pooling
        x = self.pool2(F.relu(self.conv2(x)))
        # Aplicar la tercera capa convolucional seguida de ReLU y pooling
        x = self.pool3(F.relu(self.conv3(x)))
        # Aplanar las características para la capa completamente conectada
        x = x.view(-1, 64 * 4 * 4)
        # Aplicar la primera capa completamente conectada seguida de ReLU
        x = F.relu(self.fc1(x))
        # Aplicar la capa de salida
        x = self.fc2(x)
        return x

# Crear una instancia de la red
net = CNN()

# Definir el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


## 3. Entrenamiento de modelo

#### Se itera sobre los datos de entrenamiento, se calculan las predicciones del modelo, la pérdida, y se ajustan los pesos del modelo mediante retropropagación.

In [3]:
# Número de épocas de entrenamiento
num_epochs = 10

# Bucle de entrenamiento
for epoch in range(num_epochs):  # recorrer el conjunto de datos varias veces

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Obtener las entradas y etiquetas
        inputs, labels = data

        # Poner a cero los gradientes de los parámetros
        optimizer.zero_grad()

        # Hacer forward + backward + optimizar
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Imprimir estadísticas
        running_loss += loss.item()
        if i % 100 == 99:    # imprimir cada 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Entrenamiento terminado')


Epoch [1/10], Step [100/500], Loss: 2.3041
Epoch [1/10], Step [200/500], Loss: 2.3017
Epoch [1/10], Step [300/500], Loss: 2.2993
Epoch [1/10], Step [400/500], Loss: 2.2965
Epoch [1/10], Step [500/500], Loss: 2.2923
Epoch [2/10], Step [100/500], Loss: 2.2861
Epoch [2/10], Step [200/500], Loss: 2.2719
Epoch [2/10], Step [300/500], Loss: 2.2435
Epoch [2/10], Step [400/500], Loss: 2.1861
Epoch [2/10], Step [500/500], Loss: 2.1151
Epoch [3/10], Step [100/500], Loss: 2.0564
Epoch [3/10], Step [200/500], Loss: 1.9870
Epoch [3/10], Step [300/500], Loss: 1.9590
Epoch [3/10], Step [400/500], Loss: 1.9190
Epoch [3/10], Step [500/500], Loss: 1.8916
Epoch [4/10], Step [100/500], Loss: 1.8465
Epoch [4/10], Step [200/500], Loss: 1.8417
Epoch [4/10], Step [300/500], Loss: 1.7808
Epoch [4/10], Step [400/500], Loss: 1.7554
Epoch [4/10], Step [500/500], Loss: 1.7299
Epoch [5/10], Step [100/500], Loss: 1.6909
Epoch [5/10], Step [200/500], Loss: 1.6610
Epoch [5/10], Step [300/500], Loss: 1.6375
Epoch [5/10

## 4. Evaluación de modelo

#### Se evalúa el modelo utilizando el conjunto de datos de prueba, calculando la precisión global.

In [4]:
correct = 0
total = 0

# No necesitamos calcular los gradientes para la evaluación
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión de la red en las 10000 imágenes de prueba: {100 * correct / total} %')


Precisión de la red en las 10000 imágenes de prueba: 53.33 %
